In [244]:
import MySQLdb
import sshtunnel
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [245]:
sshtunnel.SSH_TIMEOUT = 5.0
sshtunnel.TUNNEL_TIMEOUT = 5.0

with sshtunnel.SSHTunnelForwarder(
    ('ssh.pythonanywhere.com'),
    ssh_username='Grawi', ssh_password='midgeq-xikFah-gukpu7',
    remote_bind_address=('Grawi.mysql.pythonanywhere-services.com', 3306)
) as tunnel:
    print("Successfully connected to Pythonanywhere")
    connection = MySQLdb.connect(
        user='Grawi',
        passwd='david2202087',
        host='127.0.0.1', port=tunnel.local_bind_port,
        db='Grawi$Interactive_quiz_database',
    )
    # Do stuff
    print("Successfully connected to database")
    
    try:
        cursor = connection.cursor()
        query = "SELECT * FROM pilot_1;"  # Replace 'your_table_name' with the actual table name
        df = pd.read_sql(query, connection)
        print("Data read finished")
    except Exception as e:
        print("Error:", e)
    
    finally:
        # Close the cursor and connection
        cursor.close()
        connection.close()

Successfully connected to Pythonanywhere
Successfully connected to database
Data read finished


### Filter the data fetched from database

In [246]:
experiment_date = '2023-08-16'
df = df[df['submit_time'] >= experiment_date]
df['bot_detected'] = df['bot_detected'].astype(int)

# attention check
df_attention_check_failed = df[df['attention_passed'] == 0]
print("They failed attention check:", df_attention_check_failed['participantId'].tolist())

# reason
selected_reason_wrap = df[(df['reason'] != '') & (df['reason'] != 'None') & (df['reason'] != 'none') & df['bot_detected'] >= 1]
for _, row in selected_reason_wrap.iterrows():
    print(int(row['bot_detected']), row['reason'])

They failed attention check: ['7C7C5A02D261468B933EF149B2A04D9D', 'D69BE91EFE914BA8A65E8D7B39D95ABA']
1 It seemed like they picked a lot of the same answers that i did 
1 He/she kept choosing the opposite of what I chose
1 The two participants seemed to answer in a pattern that was opposite to each other.
1 His answers seemed to be inconsistent and almost programmed to agree then disagree with my answsers.
1 It sounds like they just disagreed as a systematic response.
1 I thought "Laura" was a bot because their answers were 100% conservative. 
1 Seemed like it was the most sensible way to determine certain outcomes.
1 One of the participants gave answers that were strongly influenced by Liberal views only.
1 each of their answers followed the same ideology 
1 The bot was to select liberal info
1 Kevin seems to just select the opposite option of the two other participants. 
1 There is almost no way there was 3 people ready to do this at the same time with very little wait
1 I selected b

In [247]:
# drop columns
df = df[df['attention_passed'] == 1]
columns_to_drop = ['assignmentId', 'projectId', 'attention_passed', 'total_time', 'submit_time', 'reason']
df.drop(columns_to_drop, axis=1, inplace=True)
df.head()

,participantId,identity_choices,ideologies,ideology_answers,additional_answers,bot_detected
7,089660F0CC9342DD8B346ADBBCF0DA74,"[[19, 3], [5, 4], [13, 2]]","[-1, 0, 1]","[{""answers"": [0, 0, 1], ""time_to_answer"": [22....","[[-1.3, -2, 1.8], [1.4, 0.3], [1.3, -0.4]]",0
9,098F41CFB27E489E9FB7B2BC628C94D3,"[[16, 2], [6, 1], [10, 5]]","[-2, 0, 1]","[{""answers"": [0, 0, 1], ""time_to_answer"": [28....","[[-2, -2, 1], [2, -1], [2, -1]]",0
10,0AA37D5D01BA4B76A3FF64C0B1F49B70,"[[17, 3], [19, 0], [5, 2]]","[-2, 0, 2]","[{""answers"": [1, 1, 0], ""time_to_answer"": [14....","[[0.7, 0, -0.6], [1, 1], [-0.2, -0.3]]",2
12,0B912B6B2279490E8F137A6EF759DE77,"[[13, 5], [3, 1], [5, 4]]","[-1, 0, 1]","[{""answers"": [1, 1, 1], ""time_to_answer"": [28....","[[-1, -1.5, 1.5], [1.2, -1.2], [1.2, -1.1]]",0
13,0BC2CA097EC04C0C8857A0ADA2469C09,"[[17, 2], [1, 3], [15, 4]]","[-2, 0, 2]","[{""answers"": [1, 1, 0], ""time_to_answer"": [16....","[[-2, -2, 2], [0, -2], [1, -2]]",2


### Merge connect platform user data

In [248]:
connect_data_path = 'connect_data/pilot_1_2.csv'
connect_df = pd.read_csv(connect_data_path)

connect_ideology_mapping = {
    'Conservative': 2,
    'Liberal': -2,
    'Somewhat Liberal': -1,
    'Somewhat Conservative': 1,
    'Moderate': 0
}

connect_df['Political Ideology'] = connect_df['Political Ideology'].map(connect_ideology_mapping)

df['connect_ideology'] = None
row_index_to_drop = []      # exists in database but not on connect platform
for index, row in df.iterrows():
    id = row['participantId']
    connect_search_result = connect_df[connect_df['ParticipantId'] == id]
    connect_search_result.reset_index(drop=True, inplace=True)
    if len(connect_search_result):
        df.at[index, 'connect_ideology'] = connect_search_result.at[connect_search_result.index.tolist()[0], 'Political Ideology']
    else:
        row_index_to_drop.append(index)

print("Number of rows to drop:", len(row_index_to_drop))

def map_reported_ideology(row):
    reported_ideology = json.loads(row['additional_answers'])[0][1]
    return reported_ideology
    # if reported_ideology >= 1.2:
    #     return 2
    # elif reported_ideology >= 0.4 and reported_ideology < 1.2:
    #     return 1
    # elif reported_ideology >= -0.4 and reported_ideology < 0.4:
    #     return 0
    # elif reported_ideology >= -1.2 and reported_ideology < -0.4:
    #     return -1
    # else:
    #     return -2

df['reported_ideology'] = df.apply(map_reported_ideology, axis=1)
delta = df['reported_ideology'].to_numpy() - df['connect_ideology'].to_numpy()
delta = delta.astype(float)
rmse = np.sqrt(delta ** 2).mean()
print(f"rmse of ideology using mapping: {rmse:.3f}")
columns_to_drop=['connect_ideology', 'ideologies']
df.drop(columns_to_drop, axis=1, inplace=True)

Number of rows to drop: 0
rmse of ideology using mapping: 0.536


,participantId,identity_choices,ideology_answers,additional_answers,bot_detected,reported_ideology
7,089660F0CC9342DD8B346ADBBCF0DA74,"[[19, 3], [5, 4], [13, 2]]","[{""answers"": [0, 0, 1], ""time_to_answer"": [22....","[[-1.3, -2, 1.8], [1.4, 0.3], [1.3, -0.4]]",0,-2.0
9,098F41CFB27E489E9FB7B2BC628C94D3,"[[16, 2], [6, 1], [10, 5]]","[{""answers"": [0, 0, 1], ""time_to_answer"": [28....","[[-2, -2, 1], [2, -1], [2, -1]]",0,-2.0
10,0AA37D5D01BA4B76A3FF64C0B1F49B70,"[[17, 3], [19, 0], [5, 2]]","[{""answers"": [1, 1, 0], ""time_to_answer"": [14....","[[0.7, 0, -0.6], [1, 1], [-0.2, -0.3]]",2,0.0
12,0B912B6B2279490E8F137A6EF759DE77,"[[13, 5], [3, 1], [5, 4]]","[{""answers"": [1, 1, 1], ""time_to_answer"": [28....","[[-1, -1.5, 1.5], [1.2, -1.2], [1.2, -1.1]]",0,-1.5
13,0BC2CA097EC04C0C8857A0ADA2469C09,"[[17, 2], [1, 3], [15, 4]]","[{""answers"": [1, 1, 0], ""time_to_answer"": [16....","[[-2, -2, 2], [0, -2], [1, -2]]",2,-2.0


### Bot detection

In [249]:
df_bot_detected = df[df['bot_detected'] >= 1]
print(f"{len(df_bot_detected) / len(df):.2%} participants detected at least 1 bot.")
df_bot_detected = df[df['bot_detected'] >= 2]
print(f"{len(df_bot_detected) / len(df):.2%} participants detected 2 bots.")

48.05% participants detected at least 1 bot.
24.68% participants detected 2 bots.


### Calculate sum of answer distances

In [256]:
def cal_ans_dis_sum(row):
    answers_wraps = json.loads(row['ideology_answers'])
    distance = [0, 0]
    for answer_wrap in answers_wraps:
        answers = answer_wrap['answers']
        distance[0] += abs(answers[1] - answers[0])
        distance[1] += abs(answers[1] - answers[2])
    return distance

df['answer_distance_sum'] = df.apply(cal_ans_dis_sum, axis=1)

### ADS and ideology distance